Copyright (c) 2020-2021 Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# AutoVW: ChaCha for Online AutoML with Vowpal Wabbit


## 1. Introduction


In this notebook, we use one real data example (regression task) to showcase AutoVW, which is an online AutoML solution based on the following work:

*ChaCha for online AutoML. Qingyun Wu, Chi Wang, John Langford, Paul Mineiro and Marco Rossi. To appear in ICML 2021.*

AutoVW is implemented in FLAML. FLAML requires `Python>=3.6`. To run this notebook example, please install flaml with the `notebook` option:
```bash
pip install flaml[notebook]
```

In [ ]:
!pip install flaml[notebook];
